In [34]:
import pandas as pd

geneErrorList = []

def addValuesToTable(df, colName, table):
    for index,gene in enumerate(df.index):
        if (len(gene) != len('CAGL0A00105g') ):
            geneErrorList.append(gene)
            continue
        try:
            value = df.loc[gene, colName]
            table.loc[gene, colName] = value
        except:
            geneErrorList.append(gene)
        
def getKuoValue(gene):
    start = table.loc[gene].loc["START"]
    strand = table.loc[gene].loc["STRAND"]
    ran = start - 1000
    if strand == '-':
        ran = start
        start = start + 1000
    
    # Reduce search space
    if (len(gene) != len('CAGL0A00105g') ):
        geneErrorList.append(gene)
        return
    chrom = 'chr0' + gene[5]
    search = kuo.loc[chrom]
    return search[(search.loc[:,"START"] < start) & 
                  (search.loc[:,"END"] > ran)]
    
    

In [35]:
# Get general info for each gene: location, strand, start, end
genInfo = pd.read_csv("C_glabrata_CBS138_current_feature_mochi_LocationSet.txt",
                 sep='\t',header=(0))
table = genInfo.iloc[:, [1,2,3,4]]
table.index = table.iloc[:,3]
table = table.iloc[:,[0,1,2]]

# Add roetzer data column
roetzer = pd.read_csv("Roetzer_Table_MochiView_DatasetByGene.txt", sep='\t',
                     header = (0))
roetzerYap = roetzer.iloc[:,[0,5]]
roetzerYap.index = roetzer.iloc[:,0]
roetzerYap = roetzerYap.iloc[:,[1]]

addValuesToTable(roetzerYap, "Yap1 Stressed", table)
    

In [36]:
# Add Lelandais Transcriptome Data
lelandais = pd.read_csv('TranscriptomeDataMochi.txt', sep='\t',
                       header=(0))
lelandais.index = lelandais.iloc[:, 0]
lelandais = lelandais.iloc[:, [1,3]]

for i in lelandais.columns:
    addValuesToTable(lelandais, i, table)


In [37]:
table.columns = ['START', 'END', 'STRAND', 'Roetzer log2', 'Lelandais log2 Rep1',
       'Lelandais log2 Rep2']

print(table.head(5))

                START      END STRAND  Roetzer log2  Lelandais log2 Rep1  \
FEATURE_NAME                                                               
CAGL0A00105g   2636.0   1608.0      -           NaN                  NaN   
CAGL0A00110g   4809.0   1608.0      -        0.3665                  NaN   
CAGL0A00116g   4809.0   2671.0      -           NaN                  NaN   
CAGL0A00132g  11739.0  12689.0      +           NaN                  NaN   
CAGL0A00154g  15012.0  15233.0      +           NaN                  NaN   

              Lelandais log2 Rep2  
FEATURE_NAME                       
CAGL0A00105g                  NaN  
CAGL0A00110g                  NaN  
CAGL0A00116g                  NaN  
CAGL0A00132g                  NaN  
CAGL0A00154g                  NaN  


In [38]:
import math

# Read in Kuo data
kuo = pd.read_csv('Kuo_Data_MochiView_LocationDataset.txt', sep='\t')
kuo = kuo.iloc[:, [0,1,2,3,4]]
kuo.columns = ['START', 'END', 'REP1', 'REP2', 'REP3']

for index, gene in enumerate(table.index):
    res = getKuoValue(gene)
    if type(res) == type(None):
        continue
    [rep1, rep2, rep3] = res.loc[:,["REP1","REP2","REP3"]].quantile(1)
    table.loc[gene, "Kuo log2 Rep1"] = math.log2(10**[rep1, rep2, rep3][0])
    table.loc[gene, "Kuo log2 Rep2"] = math.log2(10**[rep1, rep2, rep3][1])
    table.loc[gene, "Kuo log2 Rep3"] = math.log2(10**[rep1, rep2, rep3][2])


print(table)

                START      END STRAND  Roetzer log2  Lelandais log2 Rep1  \
FEATURE_NAME                                                               
CAGL0A00105g   2636.0   1608.0      -           NaN                  NaN   
CAGL0A00110g   4809.0   1608.0      -      0.366500                  NaN   
CAGL0A00116g   4809.0   2671.0      -           NaN                  NaN   
CAGL0A00132g  11739.0  12689.0      +           NaN                  NaN   
CAGL0A00154g  15012.0  15233.0      +           NaN                  NaN   
CAGL0A00165g  18876.0  18037.0      -     -0.119234             0.217832   
CAGL0A00187g  21757.0  19487.0      -      0.581269            -0.028898   
CAGL0A00209g  22260.0  24440.0      +     -0.002140             0.101510   
CAGL0A00231g  25241.0  27622.0      +      0.172188             0.136832   
CAGL0A00253g  28586.0  27915.0      -      0.418087            -0.014973   
CAGL0A00275g  29069.0  29869.0      +      0.195500            -0.068582   
CAGL0A00297g

In [48]:
# Read in chip-seq rep1 from Lelandais study and add to table (its already in order)

chipseq1 = pd.read_csv("LelandaisREP1log2.txt", sep='\t',header=(0))
genes = table.index[0: len(chipseq1)]

for index, gene in enumerate(genes):
    table.loc[gene, "Lelandais log2 Chip-Seq Rep1"] = chipseq1.iloc[index, 3]

print(table)

                START      END STRAND  Roetzer log2  Lelandais log2 Rep1  \
FEATURE_NAME                                                               
CAGL0A00105g   2636.0   1608.0      -           NaN                  NaN   
CAGL0A00110g   4809.0   1608.0      -      0.366500                  NaN   
CAGL0A00116g   4809.0   2671.0      -           NaN                  NaN   
CAGL0A00132g  11739.0  12689.0      +           NaN                  NaN   
CAGL0A00154g  15012.0  15233.0      +           NaN                  NaN   
CAGL0A00165g  18876.0  18037.0      -     -0.119234             0.217832   
CAGL0A00187g  21757.0  19487.0      -      0.581269            -0.028898   
CAGL0A00209g  22260.0  24440.0      +     -0.002140             0.101510   
CAGL0A00231g  25241.0  27622.0      +      0.172188             0.136832   
CAGL0A00253g  28586.0  27915.0      -      0.418087            -0.014973   
CAGL0A00275g  29069.0  29869.0      +      0.195500            -0.068582   
CAGL0A00297g

In [49]:
# table.to_csv("CurentTable.txt", sep="\t", index=True)

In [4]:
import pandas as pd
table = pd.read_csv("CurentTable.txt",
                 sep='\t',header=(0))
table.index = table.iloc[:, 0]
table = table.iloc[:, 1:]
print(table)

                START      END STRAND  Roetzer log2  Lelandais log2 Rep1  \
FEATURE_NAME                                                               
CAGL0A00105g   2636.0   1608.0      -           NaN                  NaN   
CAGL0A00110g   4809.0   1608.0      -      0.366500                  NaN   
CAGL0A00116g   4809.0   2671.0      -           NaN                  NaN   
CAGL0A00132g  11739.0  12689.0      +           NaN                  NaN   
CAGL0A00154g  15012.0  15233.0      +           NaN                  NaN   
CAGL0A00165g  18876.0  18037.0      -     -0.119234             0.217832   
CAGL0A00187g  21757.0  19487.0      -      0.581269            -0.028898   
CAGL0A00209g  22260.0  24440.0      +     -0.002140             0.101510   
CAGL0A00231g  25241.0  27622.0      +      0.172188             0.136832   
CAGL0A00253g  28586.0  27915.0      -      0.418087            -0.014973   
CAGL0A00275g  29069.0  29869.0      +      0.195500            -0.068582   
CAGL0A00297g

In [6]:
# Read in chip-seq rep2 from Lelandais study and add to table (its already in order)

chipseq2 = pd.read_csv("LelandaisREP2log2.txt", sep='\t',header=(0))
genes = table.index[0: len(chipseq2)]

for index, gene in enumerate(genes):
    table.loc[gene, "Lelandais log2 Chip-Seq Rep2"] = chipseq2.iloc[index, 3]

print(table)

                START      END STRAND  Roetzer log2  Lelandais log2 Rep1  \
FEATURE_NAME                                                               
CAGL0A00105g   2636.0   1608.0      -           NaN                  NaN   
CAGL0A00110g   4809.0   1608.0      -      0.366500                  NaN   
CAGL0A00116g   4809.0   2671.0      -           NaN                  NaN   
CAGL0A00132g  11739.0  12689.0      +           NaN                  NaN   
CAGL0A00154g  15012.0  15233.0      +           NaN                  NaN   
CAGL0A00165g  18876.0  18037.0      -     -0.119234             0.217832   
CAGL0A00187g  21757.0  19487.0      -      0.581269            -0.028898   
CAGL0A00209g  22260.0  24440.0      +     -0.002140             0.101510   
CAGL0A00231g  25241.0  27622.0      +      0.172188             0.136832   
CAGL0A00253g  28586.0  27915.0      -      0.418087            -0.014973   
CAGL0A00275g  29069.0  29869.0      +      0.195500            -0.068582   
CAGL0A00297g

In [14]:
import pickle

with open('genes.pickle', 'rb') as f:
    genes = pickle.load(f)

roetzerTargets = []
kuoTargets = []
lelandaisTargets = []

for gene in genes.keys():
    if 'lelandais' in genes[gene]:
        lelandaisTargets.append(gene)
    if 'roetzer' in genes[gene]:
        roetzerTargets.append(gene)
    if 'kuo' in genes[gene]:
        kuoTargets.append(gene)

In [27]:
for gene in lelandaisTargets:
    table.loc[gene, 'Lelandais Target'] = 1
for gene in roetzerTargets:
    table.loc[gene, 'Roetzer Target'] = 1
for gene in kuoTargets:
    table.loc[gene, 'Kuo Target'] = 1
    
# table.loc[:, "Lelandais Target"].fillna(0.0)
lelCol = table.loc[:, "Lelandais Target"].copy().fillna(0.0)
roetCol = table.loc[:, "Roetzer Target"].copy().fillna(0.0)
kuoCol = table.loc[:, "Kuo Target"].copy().fillna(0.0)
table.loc[:, 'Lelandais Target'] = lelCol
table.loc[:, 'Roetzer Target'] = roetCol
table.loc[:, 'Kuo Target'] = kuoCol
print(table)

                START      END STRAND  Roetzer log2  Lelandais log2 Rep1  \
FEATURE_NAME                                                               
CAGL0A00105g   2636.0   1608.0      -           NaN                  NaN   
CAGL0A00110g   4809.0   1608.0      -      0.366500                  NaN   
CAGL0A00116g   4809.0   2671.0      -           NaN                  NaN   
CAGL0A00132g  11739.0  12689.0      +           NaN                  NaN   
CAGL0A00154g  15012.0  15233.0      +           NaN                  NaN   
CAGL0A00165g  18876.0  18037.0      -     -0.119234             0.217832   
CAGL0A00187g  21757.0  19487.0      -      0.581269            -0.028898   
CAGL0A00209g  22260.0  24440.0      +     -0.002140             0.101510   
CAGL0A00231g  25241.0  27622.0      +      0.172188             0.136832   
CAGL0A00253g  28586.0  27915.0      -      0.418087            -0.014973   
CAGL0A00275g  29069.0  29869.0      +      0.195500            -0.068582   
CAGL0A00297g

In [28]:
table.to_csv("Table.txt", sep="\t", index=True)